In [ ]:
import mlflow.pyfunc
import pandas as pd
import time
import json
import random
import paho.mqtt.client as mqtt
import mlflow.sklearn
from prometheus_client import start_http_server, Gauge, Counter

# --- CONFIGURACIÓN DE OBSERVABILIDAD ---
# Iniciamos un servidor HTTP ligero en el puerto 8000 para que Prometheus "lea" las métricas
start_http_server(8000)

# Definimos las métricas que queremos vigilar (KPIs del texto)
DRIFT_METRIC = Gauge('model_drift_score', 'Nivel de desviación de los datos (Drift)')
PREDICTION_COUNTER = Counter('model_predictions_total', 'Total de predicciones realizadas')
MODEL_CONFIDENCE = Gauge('model_confidence', 'Confianza del modelo en la predicción')

# Simulación de Baseline (Media de temperatura conocida del entrenamiento)
BASELINE_TEMP_MEAN = 75.0

def calcular_drift(temp_actual):
    # Un cálculo simple de drift: Diferencia absoluta con la media del entrenamiento
    # En producción real usaríamos alibi-detect o Kolmogorov-Smirnov
    drift = abs(temp_actual - BASELINE_TEMP_MEAN)
    return drift

# --- CONFIGURACIÓN ---
# 1. Usamos el ID que acabas de generar en el paso anterior
RUN_ID = "f6d9982a47eb48deb02abb88b4911c3e"
MODEL_URI = f"runs:/{RUN_ID}/modelo_prensa"

# 2. Configuración MQTT (Hacia la planta)
BROKER = "ind-mqtt"
PORT = 1883
TOPIC = "planta/prediccion"

print(f"📥 Cargando el cerebro (Modelo ID: {RUN_ID}) desde MLflow...")
try:
    # Cargamos el modelo como una función genérica de Python (PyFunc)
    model = mlflow.pyfunc.load_model(MODEL_URI)
    print("✅ ¡Modelo cargado en memoria! Listo para inferencia.")
except Exception as e:
    print(f"❌ Error cargando modelo. ¿Existe el ID?: {e}")
    raise

# --- BUCLE DE SIMULACIÓN (EDGE AI) ---
client = mqtt.Client()
client.connect(BROKER, PORT, 60)

print("\n📡 Iniciando monitoreo en tiempo real...")

try:
    while True:
        # A. Generamos datos de sensor (Simulando lectura física)
        # A veces generamos valores altos para provocar la alerta
        temp_actual = round(random.uniform(60, 105), 1) # Rango peligroso > 90
        vib_actual = round(random.uniform(0.2, 1.2), 2) # Rango peligroso > 0.8
        
        # B. Preparamos los datos tal como los espera el modelo (DataFrame)
        input_data = pd.DataFrame([[temp_actual, vib_actual]], 
                                columns=['temperatura', 'vibracion'])
        
        # C. EL MODELO PIENSA (Inferencia)
        prediccion = model.predict(input_data)[0] # 0 = Ok, 1 = Fallo
        
        # D. Enviamos el resultado a la planta (Node-RED)
        estado_txt = "PELIGRO 🔴" if prediccion == 1 else "OPTIMO 🟢"
        
        payload = {
            "sensor_temp": temp_actual,
            "sensor_vib": vib_actual,
            "prediccion_ia": int(prediccion),
            "mensaje_humano": estado_txt
        }
        
        client.publish(TOPIC, json.dumps(payload))
        
        # Feedback en pantalla
        print(f"Lectura: {temp_actual}°C / {vib_actual}vib --> 🧠 IA Dice: {estado_txt}")
        
        time.sleep(2) # Esperar 2 segundos entre lecturas

except KeyboardInterrupt:
    print("🛑 Monitoreo detenido.")

/opt/conda/lib/python3.11/site-packages/pydantic/_internal/_config.py:383: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


📥 Cargando el cerebro (Modelo ID: f6d9982a47eb48deb02abb88b4911c3e) desde MLflow...


✅ ¡Modelo cargado en memoria! Listo para inferencia.

📡 Iniciando monitoreo en tiempo real...
Lectura: 68.7°C / 1.05vib --> 🧠 IA Dice: OPTIMO 🟢


/tmp/ipykernel_1296/2211912611.py:28: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client()


Lectura: 102.6°C / 1.01vib --> 🧠 IA Dice: PELIGRO 🔴
Lectura: 73.2°C / 0.47vib --> 🧠 IA Dice: OPTIMO 🟢
Lectura: 69.9°C / 0.87vib --> 🧠 IA Dice: PELIGRO 🔴
Lectura: 79.0°C / 0.83vib --> 🧠 IA Dice: OPTIMO 🟢
Lectura: 87.1°C / 0.86vib --> 🧠 IA Dice: OPTIMO 🟢
Lectura: 71.3°C / 0.33vib --> 🧠 IA Dice: OPTIMO 🟢
Lectura: 79.8°C / 0.21vib --> 🧠 IA Dice: OPTIMO 🟢
Lectura: 97.7°C / 1.18vib --> 🧠 IA Dice: PELIGRO 🔴
Lectura: 83.5°C / 0.49vib --> 🧠 IA Dice: OPTIMO 🟢
Lectura: 76.6°C / 1.07vib --> 🧠 IA Dice: OPTIMO 🟢
Lectura: 71.8°C / 0.4vib --> 🧠 IA Dice: OPTIMO 🟢
Lectura: 60.1°C / 0.97vib --> 🧠 IA Dice: OPTIMO 🟢
Lectura: 79.3°C / 1.04vib --> 🧠 IA Dice: PELIGRO 🔴
Lectura: 77.9°C / 0.76vib --> 🧠 IA Dice: OPTIMO 🟢
Lectura: 95.2°C / 0.97vib --> 🧠 IA Dice: PELIGRO 🔴
Lectura: 64.6°C / 0.73vib --> 🧠 IA Dice: OPTIMO 🟢
Lectura: 91.0°C / 0.88vib --> 🧠 IA Dice: PELIGRO 🔴
Lectura: 89.6°C / 1.0vib --> 🧠 IA Dice: PELIGRO 🔴
Lectura: 70.5°C / 0.4vib --> 🧠 IA Dice: OPTIMO 🟢
Lectura: 97.1°C / 1.14vib --> 🧠 IA Dice: PELI